<a href="https://colab.research.google.com/github/pbarizza/ML-course-2025/blob/main/pbarizza_loan_amount_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Course Machine Learning
*Professor Ladle Patel*

*Student Pietro Barizza*

*Date 27.06.2025*

**Assignment:** Develop a Simple Linear Regression Model (40% of Total Grade)

**Objective:** Your task is to develop a simple linear regression model to predict an outcome based on a dataset relevant to your chosen domain. This assignment will help you apply regression techniques, data preprocessing, and model evaluation skills.

In [180]:
#importing libraries
# pandas to load data, sklearn for the regression model, metrics and model selection
# label encoder to convert strings into numbers
# matplotlib for visualization
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [181]:
# load the dataset from github
# alternatively the dataset can be loaded locally
loan_df = pd.read_csv('https://raw.githubusercontent.com/pbarizza/ML-course-2025/refs/heads/main/LoanAmountPrediction.csv')
loan_df.head()

,Applicant_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History,Property_Area,LoanAmount
0,A001,Male,Yes,1,Graduate,Yes,2526,8734,360,1,Rural,179.0
1,A002,Female,Yes,2,Not Graduate,No,5224,845,180,1,Semiurban,134.0
2,A003,Male,Yes,0,Not Graduate,No,17069,5777,360,1,Rural,373.0
3,A004,Male,No,0,Graduate,No,13536,3036,180,1,Urban,288.0
4,A005,Male,Yes,0,Graduate,No,15547,5947,120,1,Urban,347.0


In [182]:
# get general info about the dataset
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Applicant_ID       500 non-null    object 
 1   Gender             500 non-null    object 
 2   Married            500 non-null    object 
 3   Dependents         500 non-null    object 
 4   Education          500 non-null    object 
 5   Self_Employed      500 non-null    object 
 6   ApplicantIncome    500 non-null    int64  
 7   CoapplicantIncome  500 non-null    int64  
 8   Loan_Amount_Term   500 non-null    int64  
 9   Credit_History     500 non-null    int64  
 10  Property_Area      500 non-null    object 
 11  LoanAmount         500 non-null    float64
dtypes: float64(1), int64(4), object(7)
memory usage: 47.0+ KB


In [183]:
# columns of the dataset
loan_df.columns


Index(['Applicant_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'LoanAmount'],
      dtype='object')

In [184]:
# size of the dataset
loan_df.shape

(500, 12)

In [185]:
# check for null values
loan_df.isnull().sum()

,0
Applicant_ID,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,0
ApplicantIncome,0
CoapplicantIncome,0
Loan_Amount_Term,0
Credit_History,0


In [186]:
# check for not numeric values
# columns returning False will have to be treated (encoded) first
loan_df.apply(lambda x: pd.to_numeric(x, errors='coerce').notnull().all())

,0
Applicant_ID,False
Gender,False
Married,False
Dependents,False
Education,False
Self_Employed,False
ApplicantIncome,True
CoapplicantIncome,True
Loan_Amount_Term,True
Credit_History,True


In [187]:
# since regression expects numbers to feed the model all columns that are strings mmust first be encoded as numbers
# transforming the 'Applicant_ID' into numberic value.
loan_df['Applicant_ID_num'] = loan_df['Applicant_ID'].str[1:].astype(int)
loan_df['Applicant_ID_num'].tail()

,Applicant_ID_num
495,496
496,497
497,498
498,499
499,500


In [188]:
# using label encder to transform 'Gender' into number 0=female, 1=male
loan_df['Gender'] = LabelEncoder().fit_transform(loan_df['Gender'])
loan_df['Gender'].head()


,Gender
0,1
1,0
2,1
3,1
4,1


In [189]:
# using label encder to transform 'Married' into number 0=No, 1=Yes
loan_df['Married'] = LabelEncoder().fit_transform(loan_df['Married'])
loan_df['Married'].head()


,Married
0,1
1,1
2,1
3,0
4,1


In [190]:
# transforming depends into integers. where it is 3+ making it a simple 3 by removing the '+' sign
loan_df['Dependents'] = loan_df['Dependents'].replace('3+', '3')
loan_df['Dependents'] = pd.to_numeric(loan_df['Dependents'])
loan_df['Dependents'].head()

,Dependents
0,1
1,2
2,0
3,0
4,0


In [191]:
# using label encder to transform 'Education' into number 0=No, 1=Yes
loan_df['Education'] = LabelEncoder().fit_transform(loan_df['Education'])
loan_df['Education'].head()

,Education
0,0
1,1
2,1
3,0
4,0


In [192]:
# using label encder to transform 'Self_Employed' into number 0=No, 1=Yes
loan_df['Self_Employed'] = LabelEncoder().fit_transform(loan_df['Self_Employed'])
loan_df['Self_Employed'].head()

,Self_Employed
0,1
1,0
2,0
3,0
4,0


In [193]:
# check how many distrint values are in 'Property_Area' columns
loan_df['Property_Area'].value_counts()

,count
Property_Area,
Urban,175
Semiurban,168
Rural,157


In [194]:
# using label encder to transform 'Property_Area' into numbers
# 0=Rural, 1=Semirural, 2 = Urban
loan_df['Property_Area'] = LabelEncoder().fit_transform(loan_df['Property_Area'])
loan_df['Property_Area'].head()


,Property_Area
0,0
1,1
2,0
3,2
4,2


In [195]:
# check for not numeric values
# columns returning False will have to be treated (encoded) first
loan_df.apply(lambda x: pd.to_numeric(x, errors='coerce').notnull().all())

,0
Applicant_ID,False
Gender,True
Married,True
Dependents,True
Education,True
Self_Employed,True
ApplicantIncome,True
CoapplicantIncome,True
Loan_Amount_Term,True
Credit_History,True


In [196]:
# separate independent variables (features matri X) from the dependent variable (target vector y)
X = loan_df.drop('LoanAmount', axis=1)
y = loan_df['LoanAmount']

In [197]:
# show the features
X.head()

,Applicant_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History,Property_Area,Applicant_ID_num
0,A001,1,1,1,0,1,2526,8734,360,1,0,1
1,A002,0,1,2,1,0,5224,845,180,1,1,2
2,A003,1,1,0,1,0,17069,5777,360,1,0,3
3,A004,1,0,0,0,0,13536,3036,180,1,2,4
4,A005,1,1,0,0,0,15547,5947,120,1,2,5


In [198]:
# show the target value
y.head()

,LoanAmount
0,179.0
1,134.0
2,373.0
3,288.0
4,347.0


In [199]:
# split the datasest into tran and test datasets
# using 80/20% ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
